In [1]:
import pandas as pd
from statsmodels.miscmodels.ordinal_model import OrderedModel

from drowsiness_detection.data import create_handcrafted_data

In [2]:
interval = 60
full_df = create_handcrafted_data(interval)

Extracting file ../data/potsdam_aeye_112020/001_1_a.json and response file: ../data/sleep_alc_labels/001_1_a_karolinska.csv.
Extracting file ../data/potsdam_aeye_112020/001_2_s.json and response file: ../data/sleep_alc_labels/001_2_s_karolinska.csv.
Extracting file ../data/potsdam_aeye_112020/001_3_b.json and response file: ../data/sleep_alc_labels/001_3_b_karolinska.csv.
Extracting file ../data/potsdam_aeye_112020/002_1_b.json and response file: ../data/sleep_alc_labels/002_1_b_karolinska.csv.
Extracting file ../data/potsdam_aeye_112020/002_2_a.json and response file: ../data/sleep_alc_labels/002_2_a_karolinska.csv.
Extracting file ../data/potsdam_aeye_112020/002_3_s.json and response file: ../data/sleep_alc_labels/002_3_s_karolinska.csv.
Extracting file ../data/potsdam_aeye_112020/003_1_b.json and response file: ../data/sleep_alc_labels/003_1_b_karolinska.csv.
Extracting file ../data/potsdam_aeye_112020/003_2_s.json and response file: ../data/sleep_alc_labels/003_2_s_karolinska.csv.


 55%|█████▌    | 91682/165426 [00:21<00:17, 4246.59it/s]


KeyboardInterrupt: 

In [ ]:
full_df["karolinska_response_linear_interpolation"] = full_df["karolinska_response_linear_interpolation"].astype("category")
full_df[full_df.columns[:-1]] = full_df[full_df.columns[:-1]].astype(float)


In [ ]:
full_df["karolinska_response_linear_interpolation"].unique()

In [ ]:
mod_log = OrderedModel(full_df["karolinska_response_linear_interpolation"].to_numpy(),
                       full_df[full_df.columns[:-1]].to_numpy(),
                       distr='logit')

In [ ]:
res_log = mod_log.fit(method='bfgs', disp=False)
res_log.summary()

In [ ]:
predicted = res_log.model.predict(res_log.params, exog=full_df[full_df.columns[:-1]])
predicted

In [ ]:
pred_choice = predicted.argmax(1)
print('Fraction of correct choice predictions')
print((full_df["karolinska_response_linear_interpolation"].to_numpy() == pred_choice).mean())

In [95]:
res_log

array([ 2.07353765e-03,  4.83485043e+00, -3.57397729e-01, -7.17429395e-01,
       -7.76437910e-01, -1.56366125e+00, -8.58210735e-01])

In [10]:
kss_dtype = pd.CategoricalDtype(categories=list(range(1, 10)), ordered=True)
full_df["karolinska_response_linear_interpolation"] = full_df["karolinska_response_linear_interpolation"].astype(kss_dtype)

In [43]:
df = full_df.copy().iloc[:1000]
target_df = df.pop('karolinska_response_linear_interpolation')
df.pop("frame")

0      1799
1      1800
2      1801
3      1802
4      1803
       ... 
995    2794
996    2795
997    2796
998    2797
999    2798
Name: frame, Length: 1000, dtype: int64

In [46]:
print(target_df.shape)
print(df.shape)
df = df.astype(float)

(1000,)
(1000, 7)


In [52]:
target_df = target_df.astype("category")

In [74]:
full_df.dtypes

perclos_combined                             float64
num_blinks                                   float64
mean_blink_length                            float64
mean_opening_velocity                        float64
mean_closing_velocity                        float64
max_blink_length                             float64
min_blink_length                             float64
karolinska_response_linear_interpolation    category
dtype: object

In [80]:
mod_log = OrderedModel(full_df["karolinska_response_linear_interpolation"].to_numpy(),
                       full_df[["mean_blink_length", "mean_closing_velocity"]].to_numpy(),
                       distr='logit')


In [81]:
res_log = mod_log.fit(method='nm', disp=False)
res_log.summary()

/home/tim/IM/venv/lib/python3.9/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                             OrderedModel Results                             
==============================================================================
Dep. Variable:                      y   Log-Likelihood:            -4.4973e+05
Model:                   OrderedModel   AIC:                         8.995e+05
Method:            Maximum Likelihood   BIC:                         8.995e+05
Date:                Sat, 08 Jan 2022                                         
Time:                        17:20:55                                         
No. Observations:              283986                                         
Df Residuals:                  283979                                         
Df Model:                           7                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.0020      0.000      8.665      0.000       0.002       0.002
x2             4.8649      0.061     79.535      0.000       4.745       4.985
1/2           -0.3557      0.007    -49.046      0.000      -0.370      -0.341
2/3           -0.7185      0.005   -131.957      0.000      -0.729      -0.708
3/5           -0.7764      0.005   -146.311      0.000      -0.787      -0.766
5/6           -1.5615      0.008   -194.534      0.000      -1.577      -1.546
6/7           -0.8587      0.006   -149.956      0.000      -0.870      -0.848
==============================================================================
"""